# INM702 Coursework Task 1 
### Aaron Mir (Student Number: 160001207)
#### 20/12/20

<img src="Task1(1).jpg" alt="Task1(1)" width="700"/>
<img src="Task1(2).jpg" alt="Task1(2)" width="700"/>
<img src="Task1(3).jpg" alt="Task1(3)" width="700"/>
<img src="Task1(4).jpg" alt="Task1(4)" width="700"/>


The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must  reach the lower-right corner of the grid as fast as possible. You can implement one of the two (or both, for no extra point) game modes:

    • The time spent on a cell is the number on this cell
    • The time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on

In order to solve this problem, you will provide 3 algorithms:

    • A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm using heuristics. 
    It doesn’t have to perform fast or well, but  should be better than random movements.
    • Dijkstra's algorithm
    • Ant colony optimization algorithm

You should describe the algorithms and compare them. Are they always solving the problem? How long do they take depending on the size of the maze?


In [1]:
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import animation
import numpy as np
import torch 
import torchvision as tv
import sys
import random
import copy
np.set_printoptions(threshold=sys.maxsize)
np.random.seed(1)

print(f"Python: {sys.version}")
print(f"Numpy Version: {np.__version__}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Torchvision Version: {tv.__version__}")
# Use assert to test functions
    

Python: 3.8.1 (tags/v3.8.1:1b293b6, Dec 18 2019, 23:11:46) [MSC v.1916 64 bit (AMD64)]
Numpy Version: 1.18.1
PyTorch Version: 1.6.0+cu101
Torchvision Version: 0.7.0+cu101


In [21]:
class Agent():
    '''
    A class to represent an agent. The goal of the agent is to move along the grid from the upper left corner to the lower right corner using the shortest path possible.'''

    def __init__ (self, name, maze = None):
        self.name = name
        self.i, self.j = 0, 0
        self.position = [self.i, self.j] # could just index self.position[0] and self.position[1]
        self.maze = maze
        self.isFinished = False  #Changes if agent reaches position 
        #self.marker = plt.Circle((self.__x + 0.5, self.__y + 0.5), radius=0.4, fc='y')

    def move(self, move_type):
        if move_type == 'Up':
            self.i -= 1
        if move_type == 'Down':
            self.i += 1
        if move_type == 'Left':
            self.j -= 1
        if move_type == 'Right':
            self.j += 1 
    
    def all_possible_paths(row, col, paths, maze):
        if row == len(maze) - 1:
            for i in range(col, len(maze[0])):
                paths.append(maze[row][i])
            print(paths) 
            return

        if col == len(maze[0]) - 1:
            for i in range(row, len(maze)):
                paths.append(maze[i][col])
            print(paths)
            return

        paths.append(maze[row][col])
        all_possible_paths(row + 1, col, paths[:], maze)
        all_possible_paths(row, col + 1, paths[:], maze)
    
    def path_Naive(self): #use self.maze.maze to reference the actually grid
        # Split the grid into smaller grids and compare which number on each cell is lowest
        # Here we will change the values of i and j i.e. the position of the agent
        # after the comparison is made between different cells
        # Here I will also outline the finishing point and state that the agent shouldn't go to cells it has already visited
        time = 0
        options = self.all_possible_paths(self.i, self.j, [], self.maze.maze)
        if self.isFinished == False: #use a while loop?
            for i in options:
                # use move function
                time += self.maze.maze[self.i][self.j] # finds the total amount of time travelled using Naive path

                if self.i == self.height and self.i == self.width:
                    self.isFinished == True
                    break
'''  
    def path_Dijkstra(self): # HAS TO LOOK AT THE GRID AS A WHOLE AND FIND THE PATH BEFORE TAKING IT
        time = 0
        for cell in self.maze:
            time += self.maze[self.i, self.j] # finds the total amount of time travelled using Dijkstra path

    def path_AntCol(self):
        time = 0
        for cell in self.maze:
            time += self.maze[self.i, self.j] # finds the total amount of time travelled using AntColony path
'''

In [26]:
class IntMaze():
    '''
    A class to represent a maze. A maze in this scenario is a 2D (height x width) grid where each cell has a number associated to it. This cell number indicates the time an agent spends on a cell. There are 2 game modes. Game mode 1 is where the time spent on a cell is the number on this cell. Game mode 2 is where the time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on. '''

    def __init__ (self, height, width, mode, alg, agent):  
        self.height = height
        self.width = width
        self.mode = mode
        self.agent = agent
        self.alg = alg
        self.path_find_type = {'Naive': 0, 'Dijkstra': 1, 'Ant Colony': 2}
    
    def create_maze(self): # generates the grid/map
        return np.array(np.random.randint(0, 10, size = (self.height, self.width)))

    def start_game(self):
        self.maze = self.create_maze() #create the grid/maze
        self.agent.maze = self # add agent to the maze/environment
        if self.mode == 1:
            if self.path_find_type[self.alg] == 0: # Naive approach
                self.agent.path_Naive()

            elif self.path_find_type[self.alg] == 1: # Dijkstra 
                self.agent.path_Dijkstra()

            elif self.path_find_type[self.alg] == 2: # Antcolony 
                self.agent.path_AntCol()

        if self.mode == 2:
            if self.path_find_type == 0: # Naive approach
                self.agent.path_Naive()

            elif self.path_find_type == 1: # Dijkstra 
                self.agent.path_Dijkstra()

            elif self.path_find_type == 2: # Antcolony 
                self.agent.path_AntCol()
    
    def maze_reset(self):
        pass

    def plot_maze(self): # agent has colour red which is slightly transparent? 
        # Display possible options the agent can take on the plot
        fig = plt.figure()
        cmap = colors.ListedColormap(['white','green', '#ffff99', 'gold', 'orange', '#550011', 'black', 'purple', '#f0027f', '#386cb0'])
        plt.imshow(maze, cmap = cmap, alpha = 0.5) 
        plt.colorbar(cmap=cmap)
        #use funcanimation to show agent moving along the grid?
        plt.show()   

In [11]:
class TestAgent():
    def __init__ (self, name, maze = None):
        self.name = name
        self.i, self.j = 0, 0
        self.position = (self.i, self.j) # could just index self.position[0] and self.position[1]
        self.maze = maze

    def maze_disp(self):
        print(self.maze.maze)

class TestMaze():
    def __init__ (self, height, width, mode, alg, agent):  
        self.height = height
        self.width = width
        self.mode = mode
        self.agent = agent
        self.alg = alg
        self.path_find_type = {'Naive': 0, 'Dijkstra': 1, 'AntColony': 2}
    
    def create_maze(self): # generates the grid/map
        return np.array(np.random.randint(0, 10, size = (self.height, self.width)))

    def start_game(self):
        self.maze = self.create_maze()
        self.agent.maze = self # add agent to the maze/environment
        self.agent.maze_disp()

In [12]:
test_agent = TestAgent('Bob')
maze = TestMaze(4, 9, 1, 'Naive', test_agent)
maze.start_game()

[[5 8 9 5 0 0 1 7 6]
 [9 2 4 5 2 4 2 4 7]
 [7 9 1 7 0 6 9 9 7]
 [6 9 1 0 1 8 8 3 9]]


In [9]:
test_agent.maze.maze[-1][0]

1

In [ ]:
agent_cortez = Agent('Bob')
maze = IntMaze(9, 9, 1, 'Naive', agent_cortez)
maze.start_game()
#At runtime, we want to initialise the maze, initialise the agent, choose which algorithm it will follow and what game mode it will play

In [9]:
test_maze = np.array(np.random.randint(0, 10, size = (5, 4)))

In [3]:
def all_possible_paths(row, col, paths, maze):
    if row == len(maze) - 1:
        for i in range(col, len(maze[0])):
            paths.append(maze[row][i])
        print(paths) 
        return

    if col == len(maze[0]) - 1:
        for i in range(row, len(maze)):
            paths.append(maze[i][col])
        print(paths)
        return

    paths.append(maze[row][col])
    all_possible_paths(row + 1, col, paths[:], maze)
    all_possible_paths(row, col + 1, paths[:], maze)

In [4]:
print(all_possible_paths(0, 0, [], test_maze))

[5, 0, 6, 5, 2, 4, 2]
[5, 0, 6, 9, 2, 4, 2]
[5, 0, 6, 9, 2, 4, 2]
[5, 0, 6, 9, 2, 4, 2]
[5, 0, 0, 9, 2, 4, 2]
[5, 0, 0, 9, 2, 4, 2]
[5, 0, 0, 9, 2, 4, 2]
[5, 0, 0, 1, 2, 4, 2]
[5, 0, 0, 1, 2, 4, 2]
[5, 0, 0, 1, 7, 4, 2]
[5, 8, 0, 9, 2, 4, 2]
[5, 8, 0, 9, 2, 4, 2]
[5, 8, 0, 9, 2, 4, 2]
[5, 8, 0, 1, 2, 4, 2]
[5, 8, 0, 1, 2, 4, 2]
[5, 8, 0, 1, 7, 4, 2]
[5, 8, 9, 1, 2, 4, 2]
[5, 8, 9, 1, 2, 4, 2]
[5, 8, 9, 1, 7, 4, 2]
[5, 8, 9, 5, 7, 4, 2]
None


In [11]:
test_maze

array([[4, 7, 7, 9],
       [1, 7, 0, 6],
       [9, 9, 7, 6],
       [9, 1, 0, 1],
       [8, 8, 3, 9]])

In [ ]:
'''
def immediate_possible_options(self): # this checks the possible options the agent can take at any given position. Eliminates possibility of agent from going over edges of the grid. Puts the values of each option [up, down, left, right] in an array where if one option is not possible, it is given a value of 0
        immediate_possible_options = []
        if self.i - 1 == -1 or self.i + 1 > self.maze.height:
            up = 0
            possible_options.append(up)
        else:
            up = self.maze.maze[self.i - 1][self.j]
            possible_options.append(up)
        
        if self.i + 1 > self.maze.height:
            down = 0
            possible_options.append(down)
        else:
            down = self.maze.maze[self.i + 1][self.j]
            possible_options.append(down)
        
        if self.j - 1 == -1:
            left = 0
            possible_options.append(left)
        else:
            left = self.maze.maze[self.i][self.j - 1]
            possible_options.append(left)
        
        if self.j + 1 > self.maze.height:
            right = 0
            possible_options.append(right)
        else:
            right = self.maze.maze[self.i][self.j + 1]
            possible_options.append(right)
        return possible_options
        '''